In [9]:
import pandas as pd

In [18]:
# Read the data from the CSV file
d_qml_raw = pd.read_csv('../data/raw/QAOA-Layer-Exploration_runs.csv')

In [21]:
d_qml_raw.head()
# Unpack the params column into separate columns
d_qml = pd.concat([d_qml_raw.drop(['params'], axis=1), d_qml_raw['params'].apply(eval).apply(pd.Series)], axis=1)

In [31]:
graph_features = [
    'acyclic',
    'algebraic_connectivity',
    'average_distance',
    'bipartite',
    'clique_number',
    'coefficient_of_variation',
    'connected',
    'density',
    'diameter',
    'edge_connectivity',
    'entropy',
    'eulerian',
    'first_quartile',
    'interquartile_range',
    'is_distance_regular',
    'kurtosis_weight',
    'laplacian_largest_eigenvalue',
    'laplacian_second_largest_eigenvalue',
    'maximum_degree',
    'maximum_weighted_degree',
    'mean_weight',
    'median_weight',
    'minimum_degree',
    'minimum_dominating_set',
    'minimum_weighted_degree',
    'number_of_components',
    'number_of_cut_vertices',
    'number_of_edges',
    'number_of_minimal_odd_cycles',
    'number_of_orbits',
    'number_of_vertices',
    'num_nodes',
    'planar',
    'radius',
    'range_weight',
    'ratio_of_two_largest_laplacian_eigenvaleus',
    'regular',
    'skewness_weight',
    'smallest_eigenvalue',
    'std_dev_weight',
    'third_quartile',
    'variance_weight',
    'vertex_connectivity',
    'weighted_average_clustering',
    'weighted_average_shortest_path_length',
    'weighted_diameter',
    'weighted_radius'
]
# select the graph features
d_features = d_qml[graph_features] 

In [42]:
# Read the QAOA optimal parameters for the graphs based on the run_id
def get_optimal_params(run_id):
    # find the file with the optimal parameters
    filepath = f'../optimized_parameters/{run_id}_optimized_parameters.json'
    # read the optimal parameters as json
    with open(filepath, 'r') as file:
        optimal_params
    # return the optimal parameters
    return optimal_params

get_optimal_params("5de57b42357f4015a2b713a16bcec195")


,layer,gamma,beta
0,1,[0.82381652020449],[-1.31678969756907]
1,2,"[-82.62166411505994, -5.285405489169279]","[27.140365056353623, 8.222685413023193]"
2,3,"[4.950142646626103, 2.057534389769559, 1.53552...","[0.8230515830606221, 1.423143605746311, 3.4215..."
3,4,"[0.50573359232638, 1.1736065997780911, 1.40490...","[3.5191927359669553, 3.39119390452814, 1.76726..."
4,5,"[4.135982061521808, 4.941213493782226, 1.71753...","[0.28039367856561703, 1.013215641323492, 1.841..."
5,6,"[0.44365375339217006, 1.348222413907758, -0.14...","[1.9722540785672331, 1.014776862926933, 2.4695..."
6,7,"[3.190456367992298, 1.906704257603875, 3.16063...","[0.28336953443289103, 2.783055116487109, 1.141..."
7,8,"[5.513298568391142, 1.610181764427754, 0.80355...","[0.9078180299981821, -0.35322979341076105, 0.7..."
8,9,"[5.184743938594313, 4.737202986698565, 1.13406...","[1.6194296145120441, 2.501972224397588, 1.3569..."
9,10,"[0.439840618047232, 4.328662185958829, 4.25326...","[0.267266617833241, 0.7110279823669181, 1.6070..."


In [33]:
# drop duplicates
d_features = d_features.drop_duplicates()
len(d_features)

1868

In [40]:
d_qml

,Unnamed: 0,run_id,num_nodes,metrics,tags,acyclic,algebraic_connectivity,average_distance,bipartite,clique_number,...,smallest_eigenvalue,std_dev_weight,third_quartile,variance_weight,vertex_connectivity,weighted_average_clustering,weighted_average_shortest_path_length,weighted_diameter,weighted_radius,weight_type
0,0,5de57b42357f4015a2b713a16bcec195,12,{'approx_ratio_layer_layer_1': 0.8833086835456...,"{'mlflow.runName': 'Geometric Graph', 'mlflow....",False,3.3093084761257425,1.0303030303030303,False,11,...,-3.3306690738754696e-16,0.3320582835773695,0.8610764214190368,0.11026270369234875,9,0.4206638599757642,0.20522233061316,0.5797515282324748,0.331756505097814,uniform_plus
1,1,7d6267a4900440448a833210dabd94ce,12,{'approx_ratio_layer_layer_1': 0.6541349328175...,{'mlflow.runName': 'Nearly Complete BiPartite ...,False,1.334977954054302,1.5757575757575757,False,3,...,1.7763568394002505e-15,0.18590737564467108,0.7319461584056717,0.03456155231908884,3,0.05869313270649782,0.8800456074876081,1.4240327451837975,0.90104797220365,cauchy
2,2,d4c03379594f4a0f80ef5aca37e42749,12,{'approx_ratio_layer_layer_1': 0.6317098090232...,{'mlflow.runName': 'Nearly Complete BiPartite ...,False,3.5464884426585495,1.4545454545454546,True,2,...,9.008827046283454e-16,0.16540072229574804,0.8824957815920675,0.027357398935955166,6,0.0,1.0442506578538175,1.7477436468218697,0.9602208278624915,cauchy
3,3,0753e89223e6491f91dece1df8925b64,12,{'approx_ratio_layer_layer_1': 0.6557831392034...,{'mlflow.runName': 'Watts-Strogatz small world...,False,1.0910163428152084,1.7272727272727273,False,3,...,8.881784197001252e-16,0.17400788800024733,0.8432716745048803,0.03027874508630662,3,0.18956760594180996,1.3474259988777397,2.40245609928151,1.6338968960941962,cauchy
4,4,85247eef5f60466fbb9f93fe0c815f63,12,{'approx_ratio_layer_layer_1': 0.6035858345731...,"{'mlflow.runName': 'Uniform Random Graph', 'ml...",False,0.6624207252378737,1.606060606060606,False,4,...,1.7763568394002505e-15,0.2643839538063715,0.5722952760563604,0.06989887503028956,3,0.11611440858061463,0.3812975434908506,0.654908254129088,0.4962735896260859,uniform
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909,1909,560f23661dc24e7ba8c159f4691f6ecc,12,{'approx_ratio_layer_layer_1': 0.6382353825030...,{'mlflow.runName': 'nearly_complete_bi_partite...,False,1.079540342643309,1.4545454545454546,True,2,...,8.881784197001252e-16,0.13188634166996066,0.25725123634805125,0.0173940071190856,6,0.0,0.3208777123342927,0.4924293063414674,0.28970986985792596,cauchy
1910,1910,1043f82a33a9450d87d01d8ee50d8027,12,{'approx_ratio_layer_layer_1': 0.6390226506817...,{'mlflow.runName': 'nearly_complete_bi_partite...,False,1.4786430001303958,1.4545454545454546,True,2,...,8.881784197001252e-16,0.14490312458794594,0.36071168742039783,0.020996915515349785,6,0.0,0.42075784133061356,0.6795833340185523,0.5725242795045132,cauchy
1911,1911,a179312d154847abb22500b7ce060c89,12,{'approx_ratio_layer_layer_1': 0.6444131232622...,{'mlflow.runName': 'nearly_complete_bi_partite...,False,1.03351145700025,1.4545454545454546,True,2,...,8.881784197001252e-16,0.20062722935951835,0.2771614521551742,0.04025128516047678,6,0.0,0.3024829857478813,0.6122995665137594,0.3648797795736491,cauchy
1912,1912,c3b8522ed49f4747859cdb8eb32ce3dc,12,{'approx_ratio_layer_layer_1': 0.6577538755856...,{'mlflow.runName': 'nearly_complete_bi_partite...,False,0.7731990746802507,1.4545454545454546,True,2,...,8.881784197001252e-16,0.18914015983007446,0.2502786476126664,0.03577400006054611,6,0.0,0.25089240344526526,0.40372404087969105,0.3164970963318139,cauchy
